In [1]:
import json

import numpy as np
import mlflow
import bentoml
import tensorflow as tf

2022-11-01 12:58:28.057607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 12:58:28.057627: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from src.train import load_dataset

In [3]:
*_, (X_test, y_test) = load_dataset()

In [4]:
runs = mlflow.search_runs(experiment_names=["mlflow-tensorflow-mnist"]).\
    query("status == 'FINISHED'").\
    query("`tags.mlflow.project.entryPoint` == 'train_model'").\
    sort_values("start_time", ascending=False)

In [5]:
run_id = runs.run_id.iloc[0]

model_tag = json.loads(
    mlflow.get_run(run_id).data.tags["mlflow.log-model.history"]
)
artifact_path = model_tag[0]["artifact_path"]

In [6]:
model_uri = f"runs:/{run_id}/{artifact_path}"

In [7]:
model = tf.keras.models.load_model("./models")

2022-11-01 12:58:29.233632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-01 12:58:29.233648: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-01 12:58:29.233661: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-11-01 12:58:29.233783: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model_1 = bentoml.tensorflow.save_model("keras_native", model)
print(f"BentoML model imported as keras native: {model_1}")

/opt/conda/lib/python3.9/site-packages/bentoml/_internal/utils/tensorflow.py:297: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}
2022-11-01 12:58:30.452886: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.9/site-packages/bentoml/_internal/utils/tensorflow.py:297: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  function_map = {k: getattr(m, k, None) for k in dir(m)}


INFO:tensorflow:Assets written to: /tmp/tmpg3oxdl6gbentoml_model_keras_native/assets
BentoML model imported as keras native: Model(tag="keras_native:4jxghfsz4s4gsasc")


In [9]:
model_2 = bentoml.mlflow.import_model("mlflow_keras", model_uri)
print(f"BentoML model imported as mlflow pyfunc: {model_2}")

BentoML model imported as mlflow pyfunc: Model(tag="mlflow_keras:4lvb3osz4s4gsasc")


In [10]:
loaded_model = mlflow.keras.load_model(model_uri)
model_3 = bentoml.keras.save_model("keras", loaded_model)
print(f"BentoML model imported as keras native from MLflow artifact: {model_3}")

INFO:tensorflow:Assets written to: /tmp/tmpso_o668ibentoml_model_keras/assets
BentoML model imported as keras native from MLflow artifact: Model(tag="keras:4nhrqzcz4s4gsasc")


In [11]:
pred = model.predict(X_test)

In [12]:
runner_1 = model_1.to_runner()
runner_1.init_local()

'Runner.init_local' is for debugging and testing only.


In [13]:
runner_2 = model_2.to_runner()
runner_2.init_local()

'Runner.init_local' is for debugging and testing only.
/opt/conda/lib/python3.9/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [14]:
runner_3 = model_3.to_runner()
runner_3.init_local()

'Runner.init_local' is for debugging and testing only.


In [15]:
pred1 = runner_1.run(X_test)

In [16]:
pred2 = runner_2.run(X_test)

In [17]:
pred3 = runner_3.run(X_test)